# Queue Stuff

In [1]:
import queue
class FTQueue:
    def __init__(self):
        self.Q = {}
        
    def qCreate(self, label):
        qid = self.qId(label)
        if qid==-1:
            qid = max(list(self.Q.keys())+[-1])+1
            self.Q[qid] = {'que':queue.Queue(),'label':label}
        return qid
        
    def qId(self, label):
        for k,v in self.Q.items():
            if v['label'] == label:
                return k
        return -1
    
    def qPush(self, queue_id, item):
        self.Q[queue_id]['que'].put(item)
        
    def qPop(self, queue_id):
        return self.Q[queue_id]['que'].get(False)
    
    def qTop(self, queue_id):
        return self.Q[queue_id]['que'].queue[0]
    
    def qSize(self, queue_id):
        return self.Q[queue_id]['que'].qsize()
    
    def qDestroy(self, queue_id):
        del self.Q[queue_id]

def updateQueue(ftq, commands):
    if commands[0]=='qCreate':
        label = int(commands[1])
        return ftq.qCreate(label)
        
    elif commands[0]=='qId':
        label = int(commands[1])
        return ftq.qId(label)
        
    elif commands[0]=='qPush':
        queue_id = int(commands[1])
        item = int(commands[2])
        ftq.qPush(queue_id, item)
        
    elif commands[0]=='qPop':
        queue_id = int(commands[1])
        return ftq.qPop(queue_id)
        
    elif commands[0]=='qTop':
        queue_id = int(commands[1])
        return ftq.qTop(queue_id)
        
    elif commands[0]=='qSize':
        queue_id = int(commands[1])
        return ftq.qSize(queue_id)
        
    elif commands[0]=='qDestroy':
        queue_id = int(commands[1])
        ftq.qDestroy(queue_id)

# Join group

In [2]:
import socket
import struct
import regex as re
import threading 
import os, sys

MCAST_GRP = '224.0.0.1'
MCAST_PORT = 5050
IS_ALL_GROUPS = True
IS_MACOS = False
LOCAL_HOST = True

reuse_option = socket.SO_REUSEADDR if not IS_MACOS else socket.SO_REUSEPORT
level_option = socket.IP_ADD_MEMBERSHIP if not LOCAL_HOST else socket.IP_MULTICAST_LOOP

mcast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
mcast_sock.setsockopt(socket.SOL_SOCKET, reuse_option, 1) # for macos, socket.SO_REUSEPORT, linux socket.SO_REUSEADDR

mcast_sock.bind((MCAST_GRP, MCAST_PORT))
mreq = struct.pack("4sl", socket.inet_aton(MCAST_GRP), socket.INADDR_ANY)
mcast_sock.setsockopt(socket.IPPROTO_IP, level_option, mreq) # socket.IP_MULTICAST_LOOP for the same machine

uniAddrPort = ("127.0.0.1", 8080)
ucast_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ucast_sock.bind(uniAddrPort)

In [3]:
def processCommand(command, addr):
    # if another member sends a join request to already existing group
    if command[0]=='joinRequest':
        newMemberNum = int(command[1])
        if addr not in groupMembers and addr!=uniAddrPort:
            # if number already exists denie join reques
            if newMemberNum in groupMembers.keys():
                denJoin = ('joinDenied',)
                denJoin = ",".join(map(lambda x: str(x),denJoin))
                ucast_sock.sendto(str.encode(denJoin), addr)
            # if member number does not exist accept join request
            # send conformation with your member number
            else:
                groupMembers[newMemberNum] = addr
                confJoin = ('joinConfirm', memberNumber)
                confJoin = ",".join(map(lambda x: str(x),confJoin))
                ucast_sock.sendto(str.encode(confJoin), addr)
    # collect confirmation messages sent by other group members.
    # collect member numbers and addresses.
    elif command[0]=='joinConfirm':
        if addr not in groupMembers and addr!=uniAddrPort:
            groupMembers[int(command[1])] = addr
            
    # if denied exit for now     
    elif command[0]=='joinDenied':
        print("MEMBER NUMBER {} ALREADY EXISTS. PICK ANOTHER".format(memberNumber))
        os._exit(0)
        
    # if sequence number is received for a message
    elif command[0]=='seq':
        sequences.append(command[1:])
        
    # if a msg is recived from the client
    elif command[0]=='msg': 
        messages[command[1]] = command[2:]
        

In [4]:
def mutiListner():
    '''
    Listens to group socket for messages
    '''
    print("Running Multicast Listner")
    while(multiRunning):
        data, addr = mcast_sock.recvfrom(10240)
        command = data.decode().split(',')
        print("Received {} from {}".format(command, addr))
        processCommand(command, addr)

def uniListner():
    '''
    Listens to unicast socket for messages
    '''
    print("Running Unicast Listner")
    while(uniRunning):
        data, addr = ucast_sock.recvfrom(10240)
        command = data.decode().split(',')
        print("Received {} from {}".format(command, addr))
        processCommand(command, addr)


globalSeq = 0
memberNumber = 0
groupMembers = {memberNumber:uniAddrPort}
FTq = FTQueue()

messages = {}
sequences = []

multiRunning = True
multiThread = threading.Thread(target=mutiListner, name='multicast')
multiThread.daemon = True
multiThread.start()

uniRunning = True
uniThread = threading.Thread(target=uniListner, name='unicast')
uniThread.daemon = True
uniThread.start()

Running Multicast Listner
Running Unicast Listner


In [5]:
# joining group
joinM = ('joinRequest', memberNumber)
joinM = ",".join(map(lambda x: str(x),joinM))
ucast_sock.sendto(str.encode(joinM), (MCAST_GRP, MCAST_PORT))

Received ['joinRequest', '0'] from ('127.0.0.1', 8080)


13

In [8]:
groupMembers

{0: ('127.0.0.1', 8080), 1: ('127.0.0.1', 8081)}

In [10]:
messages

{'3640974c-7dc9-444f-b822-25152673ea9d': ['qCreate', '12']}

# MiddleWare

In [ ]:
print("Received: {} from: {}".format(commands,addr))
ret = updateQueue(FTq, commands)
if ret!=None:
    print("Sending: {}".format(ret))
    sock.sendto(str.encode(str(ret)), addr)

In [ ]:
while(1):
    data, addr = sock.recvfrom(10240)
    commands = data.decode().split(',')
    print("Received: {} from: {}".format(commands,addr))
    ret = updateQueue(FTq, commands)
    if ret!=None:
        print("Sending: {}".format(ret))
        sock.sendto(str.encode(str(ret)), addr)

Received: ['qCreate', '11'] from: ('127.0.0.1', 20001)
Sending: 0
Received: ['qId', '11'] from: ('127.0.0.1', 20001)
Sending: 0
Received: ['qPush', '0', '4'] from: ('127.0.0.1', 20001)
Received: ['qTop', '0'] from: ('127.0.0.1', 20001)
Sending: 4
Received: ['qPush', '0', '6'] from: ('127.0.0.1', 20001)
Received: ['qTop', '0'] from: ('127.0.0.1', 20001)
Sending: 4
Received: ['qPop', '0'] from: ('127.0.0.1', 20001)
Sending: 6
Received: ['qTop', '0'] from: ('127.0.0.1', 20001)
Sending: 4
Received: ['qSize', '0'] from: ('127.0.0.1', 20001)
Sending: 1
Received: ['qPop', '0'] from: ('127.0.0.1', 20001)
Sending: 4
Received: ['qSize', '0'] from: ('127.0.0.1', 20001)
Sending: 0
Received: ['qTop', '0'] from: ('127.0.0.1', 20001)
